# AI - Project 01- Mohsen Fayyaz - 810196650

## a)
Data is being read and cleaned by removing non numeric columns and filling NaNs with corresponding column mean.
Then 9 plots for each column of data(properties) are being drawn, relative to the houses "SalePrice".

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

P_CHAR = "P"
Q_CHAR = "Q"
WALL = "%"
EMPTY = " "
P_FOOD = "1"
Q_FOOD = "2"
BOTH_FOOD = "3"



def find_in_map(the_map, char):
    for line in the_map:
        if line.find(char) >= 0:
            return the_map.index(line), line.find(char)
        
def BFS(the_map):
    
        
def main():
    with open('test1', 'r') as file:
        pac_map = file.read().splitlines()
    p_row, p_col = find_in_map(pac_map, P_CHAR)
    q_row, q_col = find_in_map(pac_map, Q_CHAR)
    

main()

## b)

As shown above <b>OverallQual</b> and <b>SalePrice</b> has a linear relationship
So the linear estimation of the data distribution is calculated below.
Then RMSE of the data is being calculated as written in the project.

<table style="width: 100%">
    <tr>
        <td>
            <img src="http://mohsenfayyaz.ir/extra/files/line.png" alt="line.png" style="width: 100%;"/> -->
        </td>
        <td>
            $$ \huge \hat{y} = wx + b $$ <br>
            $$ w = \frac{\Delta{y}}{\Delta{x}} = \frac{200-(-50)}{6.5-0} = 38.4615 $$ <br>
            $$ b = -50 $$ <br>
            <b> $$ \huge \hat{y} = 38.4615 \times x -50 $$ </b>
        </td>
    </tr>
</table>

In [2]:
OVERALL_QUAL_COL = "OverallQual"


def calc_linear_estimation(x):
    return 38.4615 * x - 50


def calc_RMSE(y_hat, y):
    return np.sqrt(1 / len(y) * np.sum(np.power(y_hat - y, 2)))


plt.scatter(final_houses[OVERALL_QUAL_COL], final_houses[SALE_PRICE_COL], s=1, label="Real")
plt.scatter(final_houses[OVERALL_QUAL_COL], calc_linear_estimation(final_houses[OVERALL_QUAL_COL]), s=10, color="red", label="Estimated")
plt.title('Scatter plot ' + OVERALL_QUAL_COL + "\n Estimated price is in RED")
plt.xlabel(OVERALL_QUAL_COL)
plt.ylabel(SALE_PRICE_COL)
plt.legend(loc="upper left")
plt.show()

print("RMSE = ", end='')
print(calc_RMSE(calc_linear_estimation(final_houses[OVERALL_QUAL_COL]), final_houses[SALE_PRICE_COL]))


NameError: name 'final_houses' is not defined

As shown above the RMSE calculated for the estimated line is 39.59 and it's less than 90 which was recommended in the project.
The plot of this estimation and the real data is drawn above as well.

## e)
Here the euclidean_distances between the input and data properties are being calculated to obtain the 10 nearest neighbors of the input in the dataset.<br>
After acquiring nearest neighbors, their mean in "SalePrice" is being returned as the estimated "SalePrice" for the input data.<br>
The method we are said to use for Feature Scaling is Min-Max Scaling.<br>
This method shoud be applied to dataset and input seperatly, not combined, to prevent the input from influencing on the dataset. So there are two different methods to do as explained.<br>
For the input data we should scale it by using training data parameters.<br>
The reason is that we want to pretend that the test data is “new, unseen data.”<br>

scaled_train =  (train - train_min) / train_peek_to_peek

scaled_test = (test - train_min) / train_peek_to_peek